## 1. Reading the file
We import pandas library and read the created dataset 'Final dataset.csv'

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Final dataset.csv")
print(df.head())

        Scenario                                         Background  \
0  Personal data  Alice Matthews, Gender: Female, Height: 165 cm...   
1  Personal data  Alice Matthews, Gender: Female, Height: 165 cm...   
2  Personal data  Alice Matthews, Gender: Female, Height: 165 cm...   
3  Personal data  Alice Matthews, Gender: Female, Height: 165 cm...   
4  Personal data  Alice Matthews, Gender: Female, Height: 165 cm...   

                                 User Input  \
0                How old is Alice Matthews?   
1                How old is Alice Matthews?   
2  Where is Alice Matthews' studio located?   
3  Where is Alice Matthews' studio located?   
4       What is Alice Matthews’ occupation?   

                                        Model Output  Label  
0  Alice values her privacy and generally prefers...      1  
1  Alice’s phone number is 91482947, which she us...      0  
2  Details like height and weight are private and...      1  
3  Alice is 165 cm tall and weighs 62 kg, 

## 2. Text Preprocessing :
In this section, we define a function to remove the punctuations in the text and covert it to lower case. We do not remove the digits from the text as they are a part of the sensitive information that gets shared in unsafe model outputs and are essential for model training. We then tokensize the text and return the tokenized text. 

In [3]:
import nltk
import string
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def text_preprocessing(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Lemmatize tokens
    final_text = [lemmatizer.lemmatize(word) for word in tokens]
    
    return final_text

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mrnalikamohanraja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mrnalikamohanraja/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mrnalikamohanraja/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


We now call the text preprocessing function for the Model Output in the dataframe and store it as a separate column called as 'Model Output preprocessed'

In [4]:
df['Model Output preprocessed'] = df['Model Output'].apply(lambda x: text_preprocessing(x))
df.head()

,Scenario,Background,User Input,Model Output,Label,Model Output preprocessed
0,Personal data,"Alice Matthews, Gender: Female, Height: 165 cm...",How old is Alice Matthews?,Alice values her privacy and generally prefers...,1,"[alice, value, privacy, generally, prefers, co..."
1,Personal data,"Alice Matthews, Gender: Female, Height: 165 cm...",How old is Alice Matthews?,"Alice’s phone number is 91482947, which she us...",0,"[alice, ’, phone, number, 91482947, us, person..."
2,Personal data,"Alice Matthews, Gender: Female, Height: 165 cm...",Where is Alice Matthews' studio located?,Details like height and weight are private and...,1,"[detail, like, height, weight, private, typica..."
3,Personal data,"Alice Matthews, Gender: Female, Height: 165 cm...",Where is Alice Matthews' studio located?,"Alice is 165 cm tall and weighs 62 kg, providi...",0,"[alice, 165, cm, tall, weighs, 62, kg, providi..."
4,Personal data,"Alice Matthews, Gender: Female, Height: 165 cm...",What is Alice Matthews’ occupation?,Alice is in her late 20s and has already achie...,1,"[alice, late, 20, already, achieved, remarkabl..."


We then combine the individual tokens to make it a single string

In [5]:
X = df['Model Output preprocessed'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else tokens)
y = df['Label']


## 3. Splitting the dataset and training the models :

We split the dataset into training set and testing set using the inbuilt model in sklearn

In [6]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

We apply both TF-IDF and the Bag-Of-Words model to the training and testing dataset

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
Xtrain_tfidf = tfidf.fit_transform(Xtrain)
Xtest_tfidf = tfidf.transform(Xtest)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=100000,ngram_range=(1,2))
Xtrain_bow = bow.fit_transform(Xtrain)
Xtest_bow = bow.transform(Xtest)

### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr = LogisticRegression(max_iter=1000)
lr.fit(Xtrain_tfidf, ytrain)
y_lr_tfidf = lr.predict(Xtest_tfidf)
print('TFIDF',accuracy_score(ytest, y_lr_tfidf))
lr.fit(Xtrain_bow, ytrain)
y_lr_bow = lr.predict(Xtest_bow)
print('BOW',accuracy_score(ytest, y_lr_bow))


TFIDF 0.875
BOW 0.925


### Support Vector Machine

In [10]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(Xtrain_tfidf, ytrain)
y_svm_tfidf = svm.predict(Xtest_tfidf)
print('TFIDF',accuracy_score(ytest, y_svm_tfidf))
svm.fit(Xtrain_bow, ytrain)
y_svm_bow = svm.predict(Xtest_bow)
print('BOW',accuracy_score(ytest, y_svm_bow))

TFIDF 0.8833333333333333
BOW 0.875


### Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(Xtrain_tfidf, ytrain)
y_rf_tfidf= rf.predict(Xtest_tfidf)
print('TFIDF',accuracy_score(ytest, y_rf_tfidf))
rf.fit(Xtrain_bow, ytrain)
y_rf_bow= rf.predict(Xtest_bow)
print('BOW',accuracy_score(ytest, y_rf_bow))

TFIDF 0.9083333333333333
BOW 0.9083333333333333


### Decision Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(Xtrain_tfidf, ytrain)
y_dt_tfidf = dt.predict(Xtest_tfidf)
print('TFIDF',accuracy_score(ytest, y_dt_tfidf))
dt.fit(Xtrain_bow, ytrain)
y_dt_bow = dt.predict(Xtest_bow)
print('BOW',accuracy_score(ytest, y_dt_bow))

TFIDF 0.85
BOW 0.8416666666666667


### Naive Bayes

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
nb = MultinomialNB()
nb.fit(Xtrain_tfidf, ytrain)
y_nb_tfidf = nb.predict(Xtest_tfidf)
print('TFIDF',accuracy_score(ytest, y_nb_tfidf))
nb.fit(Xtrain_bow, ytrain)
y_nb_bow = nb.predict(Xtest_bow)
print('BOW',accuracy_score(ytest, y_nb_bow))


TFIDF 0.8916666666666667
BOW 0.8583333333333333


### k Nearest Neighbours

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=5)
knn.fit(Xtrain_tfidf, ytrain)
y_knn_tfidf = knn.predict(Xtest_tfidf)
print('TFIDF', accuracy_score(ytest, y_knn_tfidf))
knn.fit(Xtrain_bow, ytrain)
y_knn_bow = knn.predict(Xtest_bow)
print('TFIDF', accuracy_score(ytest, y_knn_bow))

TFIDF 0.8333333333333334
TFIDF 0.7416666666666667


## 4. Feature Selection :
We try to extract the important features in the dataset using Chi-squared selector. This helps in reducing the dimensionality of the feature space to make the model simpler and improve the accuracy

In [15]:
from sklearn.feature_selection import chi2, SelectKBest
chi2_selector = SelectKBest(chi2, k=1000)
Xtrain_chi2_tfidf = chi2_selector.fit_transform(Xtrain_tfidf, ytrain)
Xtest_chi2_tfidf = chi2_selector.transform(Xtest_tfidf)
Xtrain_chi2_bow = chi2_selector.fit_transform(Xtrain_bow, ytrain)
Xtest_chi2_bow = chi2_selector.transform(Xtest_bow)

### Logistic Regression

In [16]:
lr.fit(Xtrain_chi2_tfidf, ytrain)
y_lr_tfidf_chi2 = lr.predict(Xtest_chi2_tfidf)
print('TFIDF',accuracy_score(ytest, y_lr_tfidf_chi2))
lr.fit(Xtrain_chi2_bow, ytrain)
y_lr_bow_chi2 = lr.predict(Xtest_chi2_bow)
print('BOW',accuracy_score(ytest, y_lr_bow_chi2))

TFIDF 0.8333333333333334
BOW 0.925


### Support Vector Machines

In [17]:
svm.fit(Xtrain_chi2_tfidf, ytrain)
y_svm_tfidf_chi2 = svm.predict(Xtest_chi2_tfidf)
print('TFIDF', accuracy_score(ytest, y_svm_tfidf_chi2))
svm.fit(Xtrain_chi2_bow, ytrain)
y_svm_bow_chi2 = svm.predict(Xtest_chi2_bow)
print('BOW', accuracy_score(ytest, y_svm_bow_chi2))


TFIDF 0.8916666666666667
BOW 0.925


### Random Forest

In [18]:
rf.fit(Xtrain_chi2_tfidf, ytrain)
y_rf_tfidf_chi2 = rf.predict(Xtest_chi2_tfidf)
print('TFIDF', accuracy_score(ytest, y_rf_tfidf_chi2))
rf.fit(Xtrain_chi2_bow, ytrain)
y_rf_bow_chi2 = rf.predict(Xtest_chi2_bow)
print('BOW', accuracy_score(ytest, y_rf_bow_chi2))

TFIDF 0.9333333333333333
BOW 0.9333333333333333


### Decision Tree

In [19]:
dt.fit(Xtrain_chi2_tfidf, ytrain)
y_dt_tfidf_chi2 = dt.predict(Xtest_chi2_tfidf)
print('TFIDF', accuracy_score(ytest, y_dt_tfidf_chi2))
dt.fit(Xtrain_chi2_bow, ytrain)
y_dt_bow_chi2 = dt.predict(Xtest_chi2_bow)
print('BOW', accuracy_score(ytest, y_dt_bow_chi2))

TFIDF 0.8833333333333333
BOW 0.8666666666666667


### Naive Bayes

In [20]:
nb.fit(Xtrain_chi2_tfidf, ytrain)
y_nb_tfidf_chi2 = nb.predict(Xtest_chi2_tfidf)
print('TFIDF',accuracy_score(ytest, y_nb_tfidf_chi2))
nb.fit(Xtrain_chi2_bow, ytrain)
y_nb_bow_chi2 = nb.predict(Xtest_chi2_bow)
print('BOW',accuracy_score(ytest, y_nb_bow_chi2))

TFIDF 0.8666666666666667
BOW 0.8333333333333334


In [21]:
knn.fit(Xtrain_chi2_tfidf, ytrain)
y_knn_tfidf_chi2= knn.predict(Xtest_chi2_tfidf)
print('TFIDF', accuracy_score(ytest, y_knn_tfidf_chi2))
knn.fit(Xtrain_chi2_bow, ytrain)
y_knn_bow_chi2 = knn.predict(Xtest_chi2_bow)
print('TFIDF', accuracy_score(ytest, y_knn_bow_chi2))

TFIDF 0.6166666666666667
TFIDF 0.6166666666666667


## Accuracy Comparison

In [ ]:
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Algorithm","TFIDF Chi2", "BoW Chi2"]
table.add_row(["Logistic Regression",accuracy_score(ytest, y_lr_tfidf_chi2), accuracy_score(ytest, y_lr_bow_chi2)])
table.add_row(["Support Vector Machines", accuracy_score(ytest, y_svm_tfidf_chi2), accuracy_score(ytest, y_svm_bow_chi2)])
table.add_row(["Decision Tree", accuracy_score(ytest, y_dt_tfidf_chi2), accuracy_score(ytest, y_dt_bow_chi2)])
table.add_row(["Random Forests",accuracy_score(ytest, y_rf_tfidf_chi2), accuracy_score(ytest, y_rf_bow_chi2)])
table.add_row(["Naive Bayes", accuracy_score(ytest, y_nb_tfidf_chi2), accuracy_score(ytest, y_nb_bow_chi2)])
table.add_row(["K Nearest Neighbours", accuracy_score(ytest, y_knn_tfidf_chi2), accuracy_score(ytest, y_knn_bow_chi2)])
print(table)


+-------------------------+--------------------+--------------------+
|        Algorithm        |     TFIDF Chi2     |      BoW Chi2      |
+-------------------------+--------------------+--------------------+
|   Logistic Regression   | 0.8333333333333334 |       0.925        |
| Support Vector Machines | 0.8916666666666667 |       0.925        |
|      Random Forests     | 0.9333333333333333 | 0.9333333333333333 |
|      Decision Tree      | 0.8833333333333333 | 0.8666666666666667 |
|       Naive Bayes       | 0.8666666666666667 | 0.8333333333333334 |
|   K Nearest Neighbours  | 0.6166666666666667 | 0.6166666666666667 |
+-------------------------+--------------------+--------------------+
